# Ejemplo de código de búsqueda con Azure Cognitive Search
Este código muestra cómo usar Azure Cognitive Search con OpenAI y el SDK de Azure Python.

## 1.- Setup inicial

### 1.1- Importacion de librerias

In [1]:
import openai
import os
from tabulate import tabulate
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector

### 1.2.- Cargar variables de entorno

In [2]:
# Cargar secretos y configuración desde el archivo .env
load_dotenv()

# OpenAI API
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))
print("OpenAI API base: {}".format(openai.api_base))
print("OpenAI API version: {}".format(openai.api_version))
print("OpenAI API type: {}".format(openai.api_type))

# Azure Search API
search_service_name = os.getenv("SEARCH_SERVICE_NAME")
search_service_key = os.getenv("SEARCH_SERVICE_KEY")
search_index_name = os.getenv("SEARCH_INDEX_NAME")
search_endpoint = "https://{}.search.windows.net/".format(search_service_name)
search_vector_config_name = os.getenv("SEARCH_VECTOR_CONFIG_NAME")
search_semantic_config_name = os.getenv("SEARCH_SEMANTIC_CONFIG_NAME")
print("Azure Search service name: {}".format(search_service_name))
print("Azure Search service key: {}".format(search_service_key[:5] + '...' + search_service_key[-5:]))
print("Azure Search index name: {}".format(search_index_name))
print("Azure Search endpoint: {}".format(search_endpoint))
print("Azure Search vector config name: {}".format(search_vector_config_name))
print("Azure Search semantic config name: {}".format(search_semantic_config_name))

result_querys = {}


OpenAI API key: 1c7f2...3613f
OpenAI API base: https://wsl-openai-canada.openai.azure.com/
OpenAI API version: 2023-03-15-preview
OpenAI API type: azure
Azure Search service name: wsl-cog-search-test-2
Azure Search service key: 9GUM9...z1hrb
Azure Search index name: test-index-10
Azure Search endpoint: https://wsl-cog-search-test-2.search.windows.net/
Azure Search vector config name: test-search-vector-config
Azure Search semantic config name: test-search-semantic-config


### 1.3.- Clase para creación de Clientes para Azure Search

In [3]:
class CreateClient(object):
    def __init__(self, endpoint, key, index_name):
        self.endpoint = endpoint
        self.index_name = index_name
        self.key = key
        self.credentials = AzureKeyCredential(key)

    # Crear un cliente de búsqueda
    # Use esto para cargar documentos al índice
    def create_search_client(self):
        return SearchClient(
            endpoint=self.endpoint,
            index_name=self.index_name,
            credential=self.credentials,
        )

    # Crear un cliente SearchIndex
    # Esto se utiliza para crear, administrar y eliminar un índice.
    def create_admin_client(self):
        return SearchIndexClient(endpoint=self.endpoint, credential=self.credentials)

### Creación de Clientes de Búsqueda de Cognitive Search

In [4]:
base_client = CreateClient(search_endpoint, search_service_key, search_index_name)
search_client = base_client.create_search_client()
admin_client = base_client.create_admin_client()

### Metodo para generación de embeddings

In [5]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(5))
def generate_embeddings(text):
    response = openai.Embedding.create(input=text, engine=embedding_model)
    embeddings = response["data"][0]['embedding']
    return embeddings

## 2.- Busqueda simple

In [6]:

results =  search_client.search(
    query_type='simple',
    query_language='es-es',
    search_text="Donde estan los indicadores de rendimiento SSCC de mayo 2020?" ,
    select=[],
    include_total_count=True
    )

result_querys["simple_query"] = {}
for i, result in enumerate(results):
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']}")

    score = result['@search.score']
    filename_and_page = f"{result['filename']} - {result['page_number']}"
    result_querys["simple_query"][i] = {"score": score, "name": filename_and_page}
    if i+1 >= 5:
        break

Page 3 of DE03599-23.pdf -> 12.440728
Page 1 of DE03599-23.pdf -> 12.182461
Page 1 of DE03746-23.pdf -> 10.699706
Page 1 of DE03612-23.pdf -> 9.795557
Page 1 of DE03741-23.pdf -> 9.021047


## 3.- Busqueda semantica

In [7]:
results =  search_client.search(
    query_type='semantic', 
    query_language='es-es', 
    semantic_configuration_name=search_semantic_config_name,
    search_text="Donde estan los indicadores de rendimiento SSCC de mayo 2020?" ,
    select=[], 
    query_caption='extractive'
    )

result_querys["semantic_query"] = {}
for i, result in enumerate(results):
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']} - {result['@search.reranker_score']}")
    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

        score = result['@search.score']
        filename_and_page = f"{result['filename']} - {result['page_number']}"
        result_querys["semantic_query"][i] = {"score": score, "name": filename_and_page}
        if i+1 >= 5:
            break

Page 1 of DE03599-23.pdf -> 12.182461 - 2.8970587253570557
Caption: los indicadores mencionados pueden ser descargados desde la siguiente ruta:    > inicio > operación > servicios complementarios > índices de desempeño y  disponibilidad de servicios complementarios > 2020 ><em> mayo 2020</em> > versión 4     cabe destacar que los indicadores de  los meses de enero  a abril y junio del año 2020, no  sufren modificaciones, …

Page 3 of DE03599-23.pdf -> 12.440728 - 2.6875765323638916
Caption: Se acoge observación   Esta observación fue recibida en el proceso de  observaciones a los Indicadores de  Desempeño de SSCC de mayo 2020, siendo  incorporada en el cálculo de los indicadores de  mayo 2020 en su versión 3 y contestada en la  carta DE05541- 22 del 5 de diciembre de 2022.

Page 1 of DE03596-23.pdf -> 6.8573756 - 1.592294692993164
Caption: los indicadores  mencionados pueden ser descargados desde la siguiente ruta:    > inicio > operación > servicios complementarios > índices de desemp

## 4.- Busqueda vectorial

In [8]:
user_query = "Donde estan los indicadores de rendimiento SSCC de mayo 2020?"
# Se genera el objeto vectorial de la consulta del usuario con el campo embeddings
vector = Vector(value=generate_embeddings(user_query), k=5, fields="embeddings")

results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=[]
    )  
    
# Muestra los resultados de la búsqueda
result_querys["vectorial_query"] = {}
for i, result in enumerate(results):
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']}")

    score = result['@search.score']
    filename_and_page = f"{result['filename']} - {result['page_number']}"
    result_querys["vectorial_query"][i] = {"score": score, "name": filename_and_page}
    if i+1 >= 5:
        break

Page 1 of DE03612-23.pdf -> 0.854698
Page 1 of DE03599-23.pdf -> 0.854264
Page 3 of DE03599-23.pdf -> 0.85401237
Page 2 of DE03733-23.pdf -> 0.851247
Page 1 of DE03596-23.pdf -> 0.8506246


## 5.- Busqueda hibrida

### 5.1.- Simple + Vector

In [9]:
results = search_client.search(
    query_type='simple',  
    search_text=user_query,  
    vectors=[vector],
    select=[]
)  

# Muestra los resultados de la búsqueda
result_querys["hybrid_query"] = {}
for i, result in enumerate(results): 
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']}")
    
    score = result['@search.score']
    filename_and_page = f"{result['filename']} - {result['page_number']}"
    result_querys["hybrid_query"][i] = {"score": score, "name": filename_and_page}
    if i+1 >= 5:
        break

Page 3 of DE03599-23.pdf -> 0.03279569745063782
Page 1 of DE03599-23.pdf -> 0.032786883413791656
Page 1 of DE03612-23.pdf -> 0.03253968432545662
Page 1 of DE03596-23.pdf -> 0.030330881476402283
Page 2 of DE03733-23.pdf -> 0.02792120911180973


### 5.2.- Semantic + Vector

In [10]:
results = search_client.search(  
    query_type='semantic',
    query_language='es-es', 
    semantic_configuration_name=search_semantic_config_name,
    query_caption='extractive',            
    search_text=user_query,  
    vectors=[vector],
    select=[]
)  

# Muestra los resultados de la búsqueda
result_querys["seman_vector_query"] = {}
for i, result in enumerate(results):
    print(f"Page {result['page_number']} of {result['filename']} -> {result['@search.score']} - {result['@search.reranker_score']}")
    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

        score = result['@search.score']
        filename_and_page = f"{result['filename']} - {result['page_number']}"
        result_querys["seman_vector_query"][i] = {"score": score, "name": filename_and_page}
        if i+1 >= 5:
            break

Page 1 of DE03599-23.pdf -> 0.032786883413791656 - 2.8970587253570557
Caption: los indicadores mencionados pueden ser descargados desde la siguiente ruta:    > inicio > operación > servicios complementarios > índices de desempeño y  disponibilidad de servicios complementarios > 2020 ><em> mayo 2020</em> > versión 4     cabe destacar que los indicadores de  los meses de enero  a abril y junio del año 2020, no  sufren modificaciones, …

Page 3 of DE03599-23.pdf -> 0.03279569745063782 - 2.6875765323638916
Caption: Se acoge observación   Esta observación fue recibida en el proceso de  observaciones a los Indicadores de  Desempeño de SSCC de mayo 2020, siendo  incorporada en el cálculo de los indicadores de  mayo 2020 en su versión 3 y contestada en la  carta DE05541- 22 del 5 de diciembre de 2022.

Page 1 of DE03596-23.pdf -> 0.030330881476402283 - 1.592294692993164
Caption: los indicadores  mencionados pueden ser descargados desde la siguiente ruta:    > inicio > operación > servicios com

### 6.- Resultados

In [11]:

header=['Document', 'Simple Query', 'Semantic Search', 'Vectorial Search', 'Hybrid Search', 'Semantic Vector Search']
table = []
length = len(next(iter(result_querys.values())))

for i in range(length):
    row = []
    for query_type in result_querys.keys():
        if not row:
            row.append(result_querys[query_type][i]["name"])  # Append document name only for the first iteration
        row.append(result_querys[query_type][i]["score"])
    table.append(row)

print(tabulate(table, headers=header, tablefmt='grid'))

NameError: name 'query_type' is not defined